In [2]:
import csv
import json
import matplotlib.pyplot as plt
import requests
import pandas as pd
from pprint import pprint
import numpy as np

response = requests.get('https://www12.statcan.gc.ca/rest/census-recensement/CR2016Geo.json?lang=E&geos=PR&topic=2&&notes=0')

b = json.loads(response.text[2:])
#pprint(b['DATA'])

prov = []
dguid=[]
for i in range((len(b['DATA']))):
    prov.append(b['DATA'][i][2])
    dguid.append(b['DATA'][i][0])

province_dguid = pd.DataFrame({'Province':prov,'dguid':dguid})
province_dguid

aboriginal = pd.DataFrame()
for id in province_dguid['dguid']:
    response2 = requests.get('https://www12.statcan.gc.ca/rest/census-recensement/CPR2016.json?lang=E&dguid='+id+'&topic=1&&notes=0')
    b2 = json.loads(response2.text[2:])
    #pprint(b2['DATA'])
    aboriginal = aboriginal.append(b2['DATA'])
    
#aboriginal

In [3]:
#abo = aboriginal.filter(items=['Total - Aboriginal identity for the population in private households - 25% sample data'])
aboriginal_t = aboriginal[aboriginal[10].str.contains('Total - Aboriginal identity for the population in private households - 25% sample data')]
aboriginal_a = aboriginal[aboriginal[10].str.contains('  Aboriginal identity')]
aboriginal_n = aboriginal[aboriginal[10].str.contains('  Non-Aboriginal identity')]
abo1 = pd.DataFrame({'Provinces':aboriginal_t[1],'Total':aboriginal_f[13]}).set_index('Provinces')
abo2 = pd.DataFrame({'Provinces':aboriginal_a[1],'Aboriginal':aboriginal_a[13]}).set_index('Provinces')
abo3 = pd.DataFrame({'Provinces':aboriginal_n[1],'Non-aboriginal':aboriginal_n[13]}).set_index('Provinces')
abo = pd.merge(abo1,abo2, on='Provinces')
abo = pd.merge(abo,abo3, on='Provinces')
abo['Aboriginal %'] = 100 * (abo['Aboriginal']/abo['Total'])
abo['Non-aboriginal %'] = 100 * (abo['Non-aboriginal']/abo['Total'])
abo

NameError: name 'aboriginal_f' is not defined

In [1]:
aboriginal.to_csv("Aboriginal Population Per Province.csv",encoding="utf-8", index=False, header=True)

NameError: name 'aboriginal' is not defined